In [48]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = [])
rob = models['free']

Building robot model from config/franka.urdf with TCP fr3_link8
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1/']
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1/', 'contact_2/']


In [51]:
from observer import EKF
rob.build_step(0.002)
rob.build_linearized()
A, C, n, y = rob.linearized(np.zeros(rob.nx), np.zeros(7), rob.inv_mass_fn(np.zeros(7)))
ekf = EKF(rob)
#%timeit ekf.step(0.01*np.ones(7), np.zeros(7), np.eye(7))
# 3x speedup from JITting the EKF step
for _ in range(100):
    ekf.step(0.01*np.ones(7), np.zeros(7))
print(ekf.x['mu'])
print(ekf.get_ext_state())

[0.01, 0.0100002, 0.01, 0.00999991, 0.01, 0.01, 0.01, 2.66617e-06, 0.00010332, 1.70459e-06, -4.1748e-05, 2.97931e-07, 1.28669e-06, 1.79647e-07]
{'q': array([0.01000001, 0.01000022, 0.01      , 0.00999991, 0.01      ,
       0.01      , 0.01      ]), 'dq': array([ 2.66617110e-06,  1.03320173e-04,  1.70458702e-06, -4.17480456e-05,
        2.97930554e-07,  1.28668970e-06,  1.79647048e-07]), 'p': DM([0.0921817, 0.00267534, 0.926045]), 'R': DM(
[[0.99975, 0.0199987, 0.00999353], 
 [0.0199957, -0.9998, 0.000399923], 
 [0.00999953, -0.000199996, -0.99995]]), 'dx': DM([7.29933e-05, 1.26815e-06, -9.6858e-06])}


In [52]:
from impedance_controller import ImpedanceController
from robot import Robot
imp = ImpedanceController(pars={'imp_stiff':[200, 200, 200], 'imp_rest':[0, 0, 0.7]}, sym_vars = [])
rob = Robot('config/franka.urdf', ctrl = imp)
rob.build_step(0.1)
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
state = {'q':q, 'dq':dq, 'M_inv':rob.inv_mass_fn(q)}
#%timeit step(**state)

for _ in range(200):
    res = rob.step(**state)
    state.update(res)
    res_dict = rob.get_ext_state(state)
    print(res_dict['p'])

#print(rob.inv_mass_fn(np.zeros(7)))


Building robot model from config/franka.urdf with TCP fr3_link8
  Building controller imp_ctrl
[0.0784591, -4.81239e-05, 0.925309]
[0.0625303, -8.68394e-05, 0.924668]
[0.0428173, -0.000101922, 0.923717]
[0.0217026, -9.6407e-05, 0.922125]
[0.00123894, -7.86271e-05, 0.919827]
[-0.0169817, -5.68147e-05, 0.91701]
[-0.0318934, -3.67549e-05, 0.913995]
[-0.0429453, -2.12586e-05, 0.911111]
[-0.0500334, -1.07249e-05, 0.908607]
[-0.0534049, -4.15016e-06, 0.906602]
[-0.053555, -9.65435e-08, 0.905096]
[-0.0511321, 2.64973e-06, 0.903999]
[-0.0468543, 4.79327e-06, 0.903178]
[-0.0414408, 6.54438e-06, 0.902492]
[-0.0355562, 7.80165e-06, 0.901825]
[-0.0297695, 8.37315e-06, 0.901094]
[-0.024528, 8.14667e-06, 0.900253]
[-0.0201451, 7.16975e-06, 0.899282]
[-0.0168012, 5.64312e-06, 0.898183]
[-0.0145558, 3.85732e-06, 0.896966]
[-0.0133669, 2.11051e-06, 0.895644]
[-0.0131158, 6.38749e-07, 0.894232]
[-0.0136326, -4.23245e-07, 0.892745]
[-0.0147214, -1.04745e-06, 0.891196]
[-0.0161818, -1.29162e-06, 0.889599]